In [ ]:
import gradio as gr
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import numpy as np

# --- 1. HÀM XỬ LÝ BACKEND NÂNG CAO ---
def advanced_prediction(median_income, housing_median_age, total_rooms, total_bedrooms,
                        population, households, latitude, longitude, ocean_proximity):

    # --- A. DỰ ĐOÁN GIÁ ---
    # (Logic y hệt như cũ)
    model_columns = X.columns
    input_df = pd.DataFrame(np.zeros((1, len(model_columns))), columns=model_columns)

    input_df['median_income'] = median_income
    input_df['housing_median_age'] = housing_median_age
    input_df['total_rooms'] = total_rooms
    input_df['total_bedrooms'] = total_bedrooms
    input_df['population'] = population
    input_df['households'] = households
    input_df['latitude'] = latitude
    input_df['longitude'] = longitude

    input_df['rooms_per_household'] = total_rooms / (households + 0.001)
    input_df['bedrooms_per_room'] = total_bedrooms / (total_rooms + 0.001)
    input_df['population_per_household'] = population / (households + 0.001)

    for col in input_df.columns:
        if 'ocean_proximity' in col: input_df[col] = 0
    if f'ocean_proximity_{ocean_proximity}' in input_df.columns:
        input_df[f'ocean_proximity_{ocean_proximity}'] = 1

    input_scaled = scaler.transform(input_df)
    predicted_price = rf_reg.predict(input_scaled)[0]

    # --- B. VẼ ĐỒNG HỒ ĐO GIÁ (GAUGE CHART) ---
    # So sánh giá dự đoán với Min/Max của dữ liệu thật
    gauge_fig = go.Figure(go.Indicator(
        mode = "gauge+number",
        value = predicted_price,
        domain = {'x': [0, 1], 'y': [0, 1]},
        title = {'text': "Định giá ($)"},
        gauge = {
            'axis': {'range': [10000, 500000], 'tickwidth': 1},
            'bar': {'color': "darkblue"},
            'steps': [
                {'range': [0, 150000], 'color': "#90EE90"},  # Rẻ (Xanh)
                {'range': [150000, 350000], 'color': "#FFD700"}, # TB (Vàng)
                {'range': [350000, 500000], 'color': "#FF6347"}], # Đắt (Đỏ)
            'threshold': {
                'line': {'color': "red", 'width': 4},
                'thickness': 0.75,
                'value': predicted_price}
        }
    ))
    gauge_fig.update_layout(height=250, margin=dict(l=10, r=10, t=30, b=10))

    # --- C. VẼ BẢN ĐỒ VỊ TRÍ (MINI MAP) ---
    # Tạo một dataframe nhỏ chỉ chứa điểm đang chọn
    map_df = pd.DataFrame({'lat': [latitude], 'lon': [longitude], 'text': [ocean_proximity]})

    map_fig = px.scatter_mapbox(map_df, lat="lat", lon="lon", hover_name="text",
                                zoom=5, height=300)
    map_fig.update_layout(mapbox_style="open-street-map")
    map_fig.update_traces(marker=dict(size=15, color='red'))
    map_fig.update_layout(margin=dict(l=0, r=0, t=0, b=0))

    return f"${predicted_price:,.2f}", gauge_fig, map_fig


# --- 2. THIẾT KẾ GIAO DIỆN (FRONTEND) ---
# Sử dụng gr.Blocks để tùy biến bố cục linh hoạt
with gr.Blocks(theme=gr.themes.Soft()) as demo:

    gr.Markdown("""
    # 🏠 AI Real Estate Valuator - California
    ### Hệ thống định giá bất động sản thông minh sử dụng Random Forest
    """)

    with gr.Row():
        # --- CỘT TRÁI: INPUT ---
        with gr.Column(scale=1):
            gr.Markdown("### 1. Thông số Vị trí")
            with gr.Row():
                lat_input = gr.Slider(32.5, 42, value=37.8, label="Vĩ độ (Bắc/Nam)")
                lon_input = gr.Slider(-124.5, -114, value=-122.2, label="Kinh độ (Đông/Tây)")

            ocean_input = gr.Dropdown(["<1H OCEAN", "INLAND", "NEAR OCEAN", "NEAR BAY", "ISLAND"],
                                      value="NEAR BAY", label="Phân loại Vị trí")

            gr.Markdown("---")
            gr.Markdown("### 2. Thông số Căn nhà & Dân cư")
            income_input = gr.Slider(0.5, 15, value=8.0, label="Thu nhập khu vực (x$10k)")
            age_input = gr.Slider(1, 52, value=30, step=1, label="Tuổi đời nhà (Năm)")

            with gr.Accordion("Chi tiết kỹ thuật (Mở rộng)", open=False):
                rooms_input = gr.Slider(100, 10000, value=2000, label="Tổng số phòng")
                bedrooms_input = gr.Slider(10, 5000, value=400, label="Tổng phòng ngủ")
                pop_input = gr.Slider(100, 5000, value=1000, label="Dân số")
                house_input = gr.Slider(10, 2000, value=300, label="Số hộ")

            predict_btn = gr.Button("🔮 ĐỊNH GIÁ NGAY", variant="primary")

        # --- CỘT PHẢI: OUTPUT ---
        with gr.Column(scale=1):
            gr.Markdown("### 📊 Kết quả Phân tích")

            # Kết quả text
            price_output = gr.Textbox(label="Giá trị ước tính",
                                      text_align="center",
                                      show_copy_button=True)

            # Đồng hồ đo
            gauge_output = gr.Plot(label="Thước đo giá trị")

            # Bản đồ
            map_output = gr.Plot(label="Vị trí trên bản đồ thực địa")

    # --- SỰ KIỆN XỬ LÝ ---
    predict_btn.click(
        fn=advanced_prediction,
        inputs=[income_input, age_input, rooms_input, bedrooms_input,
                pop_input, house_input, lat_input, lon_input, ocean_input],
        outputs=[price_output, gauge_output, map_output]
    )

    # --- DỮ LIỆU MẪU (PRESETS) ---
    # Giúp người dùng chọn nhanh kịch bản
    gr.Examples(
        examples=[
            [8.3, 41, 880, 129, 322, 126, 37.88, -122.23, "NEAR BAY"], # San Francisco (Giàu)
            [2.5, 20, 2000, 500, 1500, 400, 36.5, -119.5, "INLAND"],   # Central Valley (Nghèo)
            [5.0, 10, 3000, 600, 1000, 500, 34.0, -118.2, "NEAR OCEAN"] # Los Angeles (Ven biển)
        ],
        inputs=[income_input, age_input, rooms_input, bedrooms_input,
                pop_input, house_input, lat_input, lon_input, ocean_input],
        label="⚡ Chọn nhanh Kịch bản mẫu"
    )

# Chạy ứng dụng
demo.launch(debug=True, share=True, height=900)